In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.6 MB/s eta 0:00:00


In [3]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=c1fd91d6f543c8b94abf0be4df3907f522cbb19eb0b785fc04cffbb950a5c1dc
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm
import evaluate
from datasets import load_dataset

In [5]:
# Загрузка модели и токенизатора
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
# Загрузим модель и токенайзер
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rut5-base-absum")
model = AutoModelForSeq2SeqLM.from_pretrained("cointegrated/rut5-base-absum").to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/315 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/828k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting

config.json:   0%|          | 0.00/753 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/977M [00:00<?, ?B/s]

In [7]:
# Загружаем тестовый датасет
from datasets import load_dataset
ds = load_dataset("RussianNLP/Mixed-Summarization-Dataset")

README.md:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/538M [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/605k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/197561 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/258 [00:00<?, ? examples/s]

In [8]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'summary'],
        num_rows: 197561
    })
    test: Dataset({
        features: ['text', 'summary'],
        num_rows: 258
    })
})

In [9]:
test_data = ds['test']

In [10]:
test_data

Dataset({
    features: ['text', 'summary'],
    num_rows: 258
})

In [11]:
# Функция для суммаризации текста
def summarize(
    text, n_words=None, compression=None,
    max_length=1000, num_beams=3, do_sample=False, repetition_penalty=10.0,
    **kwargs
):
    if n_words:
        text = '[{}] '.format(n_words) + text
    elif compression:
        text = '[{0:.1g}] '.format(compression) + text
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    with torch.no_grad():
        out = model.generate(
            **x,
            max_length=max_length, num_beams=num_beams,
            do_sample=do_sample, repetition_penalty=repetition_penalty,
            **kwargs
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [12]:
# Метрика ROUGE для оценки
rouge = evaluate.load("rouge")

In [13]:
# Список для хранения предсказаний и реальных данных
predictions = []
references = []

# Генерация предсказаний и вычисление метрик
for example in tqdm(test_data):
    # Предсказание с использованием функции summarize
    prediction = summarize(example['text'])
    predictions.append(prediction)
    references.append(example['summary'])

100%|██████████| 258/258 [03:34<00:00,  1.20it/s]


In [14]:
predictions

['Определите, какой мотоцикл выбрать девушке.',
 'Что такое товародвижение в России?',
 'Для развития туризма необходимы следующие условия: – стабильная социально-экономическая ситуация (в целом, в отдельной стране и конкретном регионе).',
 'Это был язык группы германских племен, которые обитали в I веке нашей эры.',
 'Сократи текст маршрута Золотого кольца России, который прозвал искусствовед Юрий Бычков в 1967 году.',
 'Потребляйте менее 400 г (то есть пять порций) фруктов и овощей в день, кроме картофеля, маниока и других крахмалсодержащих корнеплодов.',
 'Напиши основные тезисы этого текста.',
 'Попробуйте перестраивать под себя нормальный клеточный цикл бактериальных клеток.',
 'В СПбГМТУ разработали трехмерный принтер, позволяющий печатать металлическим порошком.',
 'С кем и где будет проживать ребенок после развода родителей?',
 'Стрелковый спорт — один из древнейших прикладных видов спорта.',
 'Продолжаем рассматривать процессы оогенеза и эмбриогенеза в контексте млекопитающих 

In [17]:
references

['При выборе первого мотоцикла для девушки, следует учесть комфорт, удобство управления и доступность органов управления. Важно опробовать несколько моделей и проверить, насколько легко ставить ноги на землю и использовать тормозной рычаг. Для обучения рекомендуется выбрать мотоцикл с легкодоступными размерами и мощностью, после чего можно перейти на более крупную модель.',
 'Основной смысл данного текста заключается в том, что товародвижение играет критически важную роль в современной рыночной экономике, особенно в России. Затраты на товародвижение составляют значительную часть суммы продаж, и многие менеджеры считают его ключевым элементом маркетинга. Товародвижение влияет на рекламу, производство товаров и услуг, развитие транспортной промышленности и сферы услуг, а также на экономические отношения между предприятиями и регионами. Важность товародвижения подчеркивается его решающей ролью в обеспечении эффекта времени, пространства и владения, а также тем, что для многих покупателей 

In [16]:
# Рассчитаем метрики ROUGE
results = rouge.compute(predictions=predictions, references=references)

# Выводим результаты
print("ROUGE-1:", results["rouge1"])
print("ROUGE-2:", results["rouge2"])
print("ROUGE-L:", results["rougeL"])

ROUGE-1: 0.09766126454702756
ROUGE-2: 0.026523091848205456
ROUGE-L: 0.09658953099268683


In [20]:
# Создаем копию тестового датасета
test_data_with_predictions = test_data.to_pandas().copy()

In [21]:
test_data_with_predictions

,text,summary
0,"Кратко суммаризируй текст:\n""Если девушка выби...","При выборе первого мотоцикла для девушки, след..."
1,"Расскажи основной смысл:\n""О значении товародв...",Основной смысл данного текста заключается в то...
2,"Перескажи кратко текст:\n""Мировой опыт свидете...","Мировой опыт показывает, что для успешного раз..."
3,"Суммаризируй:\n""Готский язык был языком группы...","Готский язык - это древний германский язык, ис..."
4,"Сократи текст:\n""В состав Золотого кольца трад...",Золотое кольцо России включает 9 городов: Серг...
...,...,...
253,"Суммаризируй:\n""""Премьер-министр Харилаос Трик...",Премьер-министр Харилаос Трикупис критически о...
254,"Сократи текст:\n""Древнерусское право, действов...",Древнерусское право сильно отличалось от европ...
255,"Напиши тезисы этого текста по пунктам\n""ПАТРИО...","1. Маленький немецкий городок, известный своим..."
256,"Напиши основные тезисы этого текста\n""Все Т-кл...",1. Т-клетки происходят из гемопоэтических ство...


In [22]:
# Подставляем предсказания в колонку 'summary'
test_data_with_predictions['summary'] = predictions

In [23]:
test_data_with_predictions

,text,summary
0,"Кратко суммаризируй текст:\n""Если девушка выби...","Определите, какой мотоцикл выбрать девушке."
1,"Расскажи основной смысл:\n""О значении товародв...",Что такое товародвижение в России?
2,"Перескажи кратко текст:\n""Мировой опыт свидете...",Для развития туризма необходимы следующие усло...
3,"Суммаризируй:\n""Готский язык был языком группы...","Это был язык группы германских племен, которые..."
4,"Сократи текст:\n""В состав Золотого кольца трад...","Сократи текст маршрута Золотого кольца России,..."
...,...,...
253,"Суммаризируй:\n""""Премьер-министр Харилаос Трик...",Премьер-министр Харилаос Трикупис был резко от...
254,"Сократи текст:\n""Древнерусское право, действов...",В Московском университете был открыт Шляхетски...
255,"Напиши тезисы этого текста по пунктам\n""ПАТРИО...","Говорили, взвизгивали, махали руками"
256,"Напиши основные тезисы этого текста\n""Все Т-кл...","Напиши основные тезисы этого текста: ""Все Т-кл..."


In [ ]:
# Сохраняем датасет в CSV
test_data_with_predictions.to_csv("test_data_with_predictions.csv", index=False)